# networkxにpickleから道路データを読み込む

In [612]:
import matplotlib.pyplot as plt
import networkx as nx
import os
import numpy as np
import pandas as pd
import folium

In [613]:
# 有向グラフ作成
G = nx.DiGraph()
nx.is_directed(G)

True

In [614]:
# データ読み込み
G = nx.read_gpickle('roppongi.pickle')

In [615]:
# 中身確認
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 268
Number of edges: 576
Average in degree:   2.1493
Average out degree:   2.1493


In [616]:
# pandasに変換
p = nx.to_pandas_edgelist(G)
p.head(3)

,source,target,link,weight
0,53393500023,53393500065,"[(35.66618159246816, 139.738131446097), (35.66...",176
1,53393500023,53393502307,"[(35.66618159246816, 139.738131446097), (35.66...",61
2,53393500065,53393502294,"[(35.66536497691448, 139.7364941177584), (35.6...",13


### 最短経路

In [618]:
# 2つのノード間の最短経路を算出
v = nx.shortest_path(G, source=53393599162, target=53393578379   )

In [617]:
v # 経路ノード確認

[53393599162, 53393578379]

In [619]:
l = []  # ２つのノードに接合するエッジからlink属性を取得
for i in range(len(v)-1):
    uv = (v[i], v[i+1])
    l.extend(G.edges[uv]['link'])
l

[(35.66718981616313, 139.73841888111238),
 (35.66697318108486, 139.73865636971516),
 (35.66697318108486, 139.73865636971516),
 (35.66733148445288, 139.73909381820457),
 (35.66733148445288, 139.73909381820457),
 (35.667481470349365, 139.73921880156874),
 (35.66778144348389, 139.73951876405974),
 (35.66778144348389, 139.73951876405974),
 (35.667914754256245, 139.73925628093247),
 (35.66799807319546, 139.73908129240482),
 (35.66799807319546, 139.73908129240482),
 (35.66777309435089, 139.7388938173589)]

In [620]:
# 地図オブジェクトを生成
map_ = folium.Map(location=[center[0], center[1]], zoom_start=18, control_scale=True, prefer_canvas=True)

In [622]:
# ポリラインを置く
folium.PolyLine(locations=l, color="red", weight="4.5", opacity=0.6).add_to(map_)

In [621]:
# 表示
map_

## 道路データを編集

In [624]:
# 先程のsource→targetの一方通行を逆方向にも通れるように

# まずエッジ（source → target）が無いことを確認（KeyError）
G[53393599162][53393578379]

{'weight': 80,
 'link': [(35.667189, 139.738418),
  (35.667383, 139.738571),
  (35.66757, 139.73873),
  (35.667773, 139.738893)]}

In [623]:
# 新たに逆方向の道を追加
new_way = [(35.667189, 139.738418), (35.667383, 139.738571), (35.667570, 139.738730), (35.667773, 139.738893)]
G.add_edge(53393599162, 53393578379, weight=80)
G[53393599162][53393578379]['link'] = new_way

In [626]:
# 確認
G[53393599162][53393578379]

{'weight': 80,
 'link': [(35.667189, 139.738418),
  (35.667383, 139.738571),
  (35.66757, 139.73873),
  (35.667773, 139.738893)]}

In [625]:
# 経路探索
v = nx.shortest_path(G, source=53393599162, target=53393578379   )

In [627]:
# ポリラインを置く（青）
l = []
for i in range(len(v)-1):
    uv = (v[i], v[i+1])
    l.extend(G.edges[uv]['link'])
folium.PolyLine(locations=l, color="blue", weight="4.5", opacity=0.6).add_to(map_)

In [629]:
# 再度表示
map_

In [628]:
# グラフを保存
nx.write_gpickle(g, 'roppongi.pickle')